In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-30 17:05:57--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.4’

minsearch.py.4      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-30 17:05:57 (16.5 MB/s) - ‘minsearch.py.4’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
index = minsearch.Index(
     text_fields=["question", "question", "section"],
     keyword_fields=["course"]
 )

In [7]:
index.fit(documents)

In [8]:
from openai import OpenAI

In [9]:
client = OpenAI()

In [23]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=10
    )

    return results

In [17]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from FAQ database. 
Use only facts from the CONTEXT when answering the QUESTON. 

QUESTION: {question}

CONTEXT: 
{context}
""".strip()
    
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [18]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [26]:
query = "How do I run kafka?"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [27]:
rag(query)

'To run Kafka, follow the specific instructions based on the language you are using:\n\n**For Java Kafka:**\n1. Navigate to the project directory.\n2. Run the following command in the terminal:\n   ```sh\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\nReplace `<jar_name>` with the actual name of your JAR file.\n\n**For Python Kafka:**\n1. Create and activate a virtual environment and install the required packages:\n   ```sh\n   python -m venv env\n   source env/bin/activate\n   pip install -r ../requirements.txt\n   ```\n   If you are on Windows, use `env\\Scripts\\activate` instead of `source env/bin/activate`.\n\n2. To deactivate the virtual environment after use, run:\n   ```sh\n   deactivate\n   ```\n\nMake sure Docker images for Kafka are up and running if you are using Docker.\n\nBy following these steps, you should be able to run Kafka producers, consumers, and streams in the terminal.'

In [28]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course even after it has started. You are eligible to submit the homework even if you're not officially registered. However, please be mindful of the deadlines for turning in your final projects and try not to leave everything to the last minute."